# Bar in NYC (Manhattan, Brooklyn), Richmond VA, Seattle
Film production houses, film rental houses

Beer, fresh out of college drinks
- microbrew
- need fine options on food menu
- shitty vodka is best vodka

- microwinery <3 

_Target customers_<br>
- Middle-aged
- Mid-20s hipsters

Restaurant
SPORTS BAR that is always closed




Add location to a map through the import we learned on learn.co

# USE BASE DATA

First step is to get requests:

In [351]:
import requests
import json
import sys
import pandas as pd
print(sys.path)

['/Users/jakeash/Desktop/testing/Yelp_Jake_Mike_Phase_1', '/Users/jakeash/opt/anaconda3/envs/learn-env/lib/python38.zip', '/Users/jakeash/opt/anaconda3/envs/learn-env/lib/python3.8', '/Users/jakeash/opt/anaconda3/envs/learn-env/lib/python3.8/lib-dynload', '', '/Users/jakeash/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages', '/Users/jakeash/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/extensions', '/Users/jakeash/.ipython']


In [352]:
url =  'https://api.yelp.com/v3/businesses/search' #points to url from my own yelp developer page

In [353]:
from  keys  import  client_id, api_key

In [354]:
print(api_key)

iRaRIQA2fkcxjxPGmsFU1pH4DHRuizbjxffWPV2mDgDCgBVPC9Cn0DXzX8jvCK9RRXLbjgXij4E2eN6AlJvx7L6Mhquc2cWaNe2NbvDSwhSesMfI1FrZHmNV55kmYHYx


In [355]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }


In [356]:
### THIS CAN BE DYNAMIC
# what type of business do you want to search
#term = 'Film Production' # this would 
term = 'Bar'
#where do you want to perform this search
location = 'Brooklyn'

# what is your other parameter you want to search against
#categories = 'Video/Film'
categories = 'Restaurant'

In [357]:

url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
    
            }


In [358]:
url_params

{'term': 'Bar',
 'location': 'Brooklyn',
 'categories': 'Restaurant',
 'limit': 50}

In [364]:
response = requests.get(url, headers=headers, params=url_params)
json.loads(response.text)
response.json()
data = response.json()
print(data['total'], data.keys())
# print(response)

6200 dict_keys(['businesses', 'total', 'region'])


In [329]:
def yelp_call(counter, headers, url_params): 
    response = requests.get(url, headers=headers, params=url_params) # our url, header and params should be consistent, atleast with our Yelp data
    return response.json()

In [330]:
yelp_call(0)

{'businesses': [{'id': 'iG5zNvIQG8v4kqbzG_S_gw',
   'alias': 'the-rogers-garden-brooklyn',
   'name': 'The Rogers Garden',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/58Z6dkgBe_4yzTgOeIYcsQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/the-rogers-garden-brooklyn?adjust_creative=dCp2ArWfqcaObE2KLuaGww&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=dCp2ArWfqcaObE2KLuaGww',
   'review_count': 45,
   'categories': [{'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.65424833, 'longitude': -73.952965},
   'transactions': ['delivery'],
   'location': {'address1': '708 Rogers Ave',
    'address2': None,
    'address3': '',
    'city': 'Brooklyn',
    'zip_code': '11226',
    'country': 'US',
    'state': 'NY',
    'display_address': ['708 Rogers Ave', 'Brooklyn, NY 11226']},
   'phone': '+13472953866',
   'display_phone': '(347) 295-3866',
   'distance': 1177.8069659015932},
  {'id':

In [348]:
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000: #### MIKE NOTE I believe num should be referring to the number of results we have in our dataframe
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(num, headers, url_params)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    df_save(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

NameError: name 'num' is not defined

#### Parsing the Data

In [331]:
business = data['businesses'][44]

In [332]:

# name, rating, review_count, price, ##location?

In [333]:
business['price']

'$$'

In [334]:
business['location']['city']

'Brooklyn'

In [335]:
full_address = business['location']['address1'], business['location']['city'], business['location']['state'], business['location']['zip_code']
full_address

('555 7th Ave', 'Brooklyn', 'NY', '11215')

In [336]:
business_rating = business['rating']
business_rating

3.5

In [337]:
biz_tuple = (business['name'], business['location']['display_address'], business['rating'])

In [338]:
biz_tuple

('Greenwood Park', ['555 7th Ave', 'Brooklyn, NY 11215'], 3.5)

In [339]:
def parse_data(list_of_data):
    # create a container to hold our parsed data
    businesses = []
    # loop through our business and 
    for business in list_of_data:
        biz_price = None
        if 'price' not in business.keys():
            biz_price = None
        else:
            biz_price = len(business['price'])
    # parse each individual business into a tuple
        biz_tuple = (business['name'],
                     business['location']['display_address'],
                     business['location']['city'],
                     business['rating'],
                     business['review_count'],
                     business['coordinates'],
                     biz_price)
    # add each individual business tuple to our data container
        businesses.append(biz_tuple)
    # return the container with all of the parsed results
    return businesses

## Make a function that adds a price column, and if the dict doesnt have a price column, add a vlaue of Null

In [340]:
parsed_results = parse_data(data['businesses'])

In [347]:
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    df_save(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 50

NameError: name 'num' is not defined

In [341]:
parsed_results # list of tuples representing name, address, and rating of establishments

[('The Rogers Garden',
  ['708 Rogers Ave', 'Brooklyn, NY 11226'],
  'Brooklyn',
  4.5,
  45,
  {'latitude': 40.65424833, 'longitude': -73.952965},
  None),
 ('Flatbush Zombie House',
  ['734 Flatbush Ave', 'Brooklyn, NY 11226'],
  'Brooklyn',
  4.0,
  102,
  {'latitude': 40.6550392260637, 'longitude': -73.9600387215614},
  2),
 ('End Zone Sports Bar & Lounge',
  ['28 Dooley St', 'Brooklyn, NY 11235'],
  'Brooklyn',
  5.0,
  6,
  {'latitude': 40.58437, 'longitude': -73.9453},
  None),
 ('Boobie Trap',
  ['308 Bleecker St', 'Brooklyn, NY 11237'],
  'Brooklyn',
  4.0,
  273,
  {'latitude': 40.7003517876193, 'longitude': -73.9161294247438},
  1),
 ('The Crown Inn',
  ['724 Franklin Ave', 'Brooklyn, NY 11238'],
  'Brooklyn',
  4.0,
  200,
  {'latitude': 40.673874, 'longitude': -73.957101},
  2),
 ('The Wilky',
  ['108 Patchen Ave', 'Brooklyn, NY 11221'],
  'Brooklyn',
  4.5,
  147,
  {'latitude': 40.68802, 'longitude': -73.92735},
  2),
 ('MOON BAR Rooftop',
  ['137-72 Northern Blvd', 'Fl 

TypeError: unhashable type: 'list'

In [342]:
pd.DataFrame(parsed_results)

,0,1,2,3,4,5,6
0,The Rogers Garden,"[708 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,4.5,45,"{'latitude': 40.65424833, 'longitude': -73.952...",NaN
1,Flatbush Zombie House,"[734 Flatbush Ave, Brooklyn, NY 11226]",Brooklyn,4.0,102,"{'latitude': 40.6550392260637, 'longitude': -7...",2.0
2,End Zone Sports Bar & Lounge,"[28 Dooley St, Brooklyn, NY 11235]",Brooklyn,5.0,6,"{'latitude': 40.58437, 'longitude': -73.9453}",NaN
3,Boobie Trap,"[308 Bleecker St, Brooklyn, NY 11237]",Brooklyn,4.0,273,"{'latitude': 40.7003517876193, 'longitude': -7...",1.0
4,The Crown Inn,"[724 Franklin Ave, Brooklyn, NY 11238]",Brooklyn,4.0,200,"{'latitude': 40.673874, 'longitude': -73.957101}",2.0
5,The Wilky,"[108 Patchen Ave, Brooklyn, NY 11221]",Brooklyn,4.5,147,"{'latitude': 40.68802, 'longitude': -73.92735}",2.0
6,MOON BAR Rooftop,"[137-72 Northern Blvd, Fl 9, Flushing, NY 11354]",Flushing,4.0,27,"{'latitude': 40.7635, 'longitude': -73.82844}",NaN
7,Sally’s,"[151 Tompkins Ave, Brooklyn, NY 11206]",Brooklyn,4.5,16,"{'latitude': 40.69413, 'longitude': -73.94592}",NaN
8,Margarita Island,"[1105 Bowery St, Brooklyn, NY 11224]",Brooklyn,3.5,38,"{'latitude': 40.57485, 'longitude': -73.97922}",2.0
9,Bearded Lady,"[686A Washington Ave, Brooklyn, NY 11238]",Brooklyn,4.0,193,"{'latitude': 40.6774605398157, 'longitude': -7...",2.0


In [343]:
df = pd.DataFrame(parsed_results, columns=['Name', 'Address','City', 'Rating','Review Count','Coordinates','Price'])

In [344]:
df

,Name,Address,City,Rating,Review Count,Coordinates,Price
0,The Rogers Garden,"[708 Rogers Ave, Brooklyn, NY 11226]",Brooklyn,4.5,45,"{'latitude': 40.65424833, 'longitude': -73.952...",NaN
1,Flatbush Zombie House,"[734 Flatbush Ave, Brooklyn, NY 11226]",Brooklyn,4.0,102,"{'latitude': 40.6550392260637, 'longitude': -7...",2.0
2,End Zone Sports Bar & Lounge,"[28 Dooley St, Brooklyn, NY 11235]",Brooklyn,5.0,6,"{'latitude': 40.58437, 'longitude': -73.9453}",NaN
3,Boobie Trap,"[308 Bleecker St, Brooklyn, NY 11237]",Brooklyn,4.0,273,"{'latitude': 40.7003517876193, 'longitude': -7...",1.0
4,The Crown Inn,"[724 Franklin Ave, Brooklyn, NY 11238]",Brooklyn,4.0,200,"{'latitude': 40.673874, 'longitude': -73.957101}",2.0
5,The Wilky,"[108 Patchen Ave, Brooklyn, NY 11221]",Brooklyn,4.5,147,"{'latitude': 40.68802, 'longitude': -73.92735}",2.0
6,MOON BAR Rooftop,"[137-72 Northern Blvd, Fl 9, Flushing, NY 11354]",Flushing,4.0,27,"{'latitude': 40.7635, 'longitude': -73.82844}",NaN
7,Sally’s,"[151 Tompkins Ave, Brooklyn, NY 11206]",Brooklyn,4.5,16,"{'latitude': 40.69413, 'longitude': -73.94592}",NaN
8,Margarita Island,"[1105 Bowery St, Brooklyn, NY 11224]",Brooklyn,3.5,38,"{'latitude': 40.57485, 'longitude': -73.97922}",2.0
9,Bearded Lady,"[686A Washington Ave, Brooklyn, NY 11238]",Brooklyn,4.0,193,"{'latitude': 40.6774605398157, 'longitude': -7...",2.0


In [345]:
df.groupby('City')['Name'].count()

City
Brooklyn                     45
Carroll Gardens               1
Far Rockaway                  1
Flushing                      1
Prospect Lefferts Gardens     2
Name: Name, dtype: int64

In [346]:
'''
Arverne              1
Astoria              2
Bronx                1
Brooklyn            14
Hoboken              1
Jersey City          1
Long Island City     4
Middle Village       1
New York            20
Queens               5
'''

'\nArverne              1\nAstoria              2\nBronx                1\nBrooklyn            14\nHoboken              1\nJersey City          1\nLong Island City     4\nMiddle Village       1\nNew York            20\nQueens               5\n'